In [77]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

## Data Preprocessing

In [78]:
train_df = pd.read_json('train.jsonl', lines=True)
X_train = train_df['string']
y_train = train_df['label']

test_df = pd.read_json('test.jsonl', lines=True)
X_test = test_df['string']
y_test = test_df['label']

train_df

,source,citeEnd,sectionName,citeStart,string,label,label_confidence,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label2_confidence
0,explicit,175.0,Introduction,168.0,"However, how frataxin interacts with the Fe-S ...",background,1.0000,1872080baa7d30ec8fb87be9a65358cd3a7fb649,894be9b4ea46a5c422e81ef3c241072d4c73fdc0,True,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,11,NaN,NaN
1,explicit,36.0,Novel Quantitative Trait Loci for Seminal Root...,16.0,"In the study by Hickey et al. (2012), spikes w...",background,1.0000,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b,b6642e19efb8db5623b3cc4eef1c5822a6151107,True,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,2,NaN,NaN
2,explicit,228.0,Introduction,225.0,"The drug also reduces catecholamine secretion,...",background,1.0000,9cdf605beb1aa1078f235c4332b3024daa8b31dc,4e6a17fb8d7a3cada601d942e22eb5da6d01adbd,False,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,0,NaN,NaN
3,explicit,110.0,Discussion,46.0,By clustering with lowly aggressive close kin ...,background,1.0000,d9f3207db0c79a3b154f3875c9760cc6b056904b,2cc6ff899bf17666ad35893524a4d61624555ed7,False,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,3,NaN,NaN
4,explicit,239.0,Discussion,234.0,Ophthalmic symptoms are rare manifestations of...,background,1.0000,88b86556857f4374842d2af2e359576806239175,a5bb0ff1a026944d2a47a155462959af2b8505a8,False,88b86556857f4374842d2af2e359576806239175>a5bb0...,88b86556857f4374842d2af2e359576806239175>a5bb0...,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8238,explicit,50.0,,28.0,"Importantly, the results of Pascalis et al. (2...",background,0.7350,6f68ccd37718366c40ae6aeedf0b935bf560b215,60ed4bdabf92b2fbd6162dbd8979888cccca55d7,True,6f68ccd37718366c40ae6aeedf0b935bf560b215>60ed4...,6f68ccd37718366c40ae6aeedf0b935bf560b215>60ed4...,15,NaN,NaN
8239,explicit,182.0,DISCUSSION,179.0,"As suggested by Nguena et al, there is a need ...",background,0.7508,f2a1c1704f9587c94ed95bc98179dc499e933f5e,574e659da7f6c62c07bfaaacd1f31d65bd75524c,True,f2a1c1704f9587c94ed95bc98179dc499e933f5e>574e6...,f2a1c1704f9587c94ed95bc98179dc499e933f5e>574e6...,1,NaN,NaN
8240,explicit,120.0,DISCUSSION,108.0,Skeletal muscle is also a primary site of dise...,background,1.0000,18c97ea2ff60c110cc2a523e0fdf729608cbb083,fc13b9c3dfcc121013edaa12fa8ce7842aaed21a,False,18c97ea2ff60c110cc2a523e0fdf729608cbb083>fc13b...,18c97ea2ff60c110cc2a523e0fdf729608cbb083>fc13b...,8,NaN,NaN
8241,explicit,221.0,,185.0,ACTIVATION OF TRANSCRIPTION FACTORS Roles for ...,method,NaN,4ec9b89857c0b27e8a4bd3745b7358f387773527,81affdba19e38e2b17cf7b9e93792cc2028cf21d,True,4ec9b89857c0b27e8a4bd3745b7358f387773527>81aff...,4ec9b89857c0b27e8a4bd3745b7358f387773527>81aff...,0,NaN,NaN


In [79]:
def cleaning(text):
    stop_words = stopwords.words('english')
    text = text.lower()
    text = ' '.join(x for x in text.split() if x not in stop_words)
    return text

In [80]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = []
    for x in text.split():
        x = lemmatizer.lemmatize(x)
        words.append(x)
    text = ' '.join(words)
    return text

In [81]:
def preprocessing(text):
    # Tokenization
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+')
    text = cleaning(text)
    text = lemmatize(text)
    text = ' '.join(tokenizer.tokenize(text))
    return text

In [82]:
def augment_data_multiclass(X, y):
    df = pd.concat([X, y], axis=1)
    majority_class_size = df['label'].value_counts().max()
    upsampled_dataframes = []
    for class_label in df['label'].unique():
        class_df = df[df['label'] == class_label]
        if len(class_df) < majority_class_size:
            class_df_upsampled = resample(class_df, replace=True, n_samples=majority_class_size, random_state=10)
            upsampled_dataframes.append(class_df_upsampled)
        else:
            upsampled_dataframes.append(class_df)
    upsampled_df = pd.concat(upsampled_dataframes)
    return upsampled_df['string'], upsampled_df['label']

## Feature Extraction

### BOW Representation

In [83]:
#X_train, y_train = augment_data_multiclass(X_train, y_train)
cv = CountVectorizer(ngram_range=(1,1), preprocessor=preprocessing)
X_train_bow = cv.fit_transform(X_train)
X_test_bow = cv.transform(X_test)

### TF-IDF Representation

In [84]:
#X_train, y_train = augment_data_multiclass(X_train, y_train)
vectorizer = TfidfVectorizer(preprocessor=preprocessing, ngram_range=(1, 2), min_df=3, max_df=0.5, use_idf=True, smooth_idf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

## Model Construction and Tuning

### Implementing Naive Bayes

Using the preprocessed data, we are able to run and process a simple Naive Bayes model. First, initialize the GaussianNB model.

In [85]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import GridSearchCV

In [86]:
# Calculating priors (P(y) = #documents of class y/#total documents)
def get_prior(class_name: str):
    class_docs_count = y_train.value_counts()[class_name]
    total_docs = y_train.count()

    return class_docs_count / total_docs

In [87]:
classes = y_train.unique()

priors = [get_prior(class_name) for class_name in classes]

In [88]:
# model = GaussianNB()
# parameters = {
#     'priors': [priors, [0.5, 0.3, 0.2], [0.5, 0.2, 0.3], [0.6, 0.2, 0.2], [0.4, 0.3, 0.3]],
#     'var_smoothing': [1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]
# }

1. BoW Representation

In [89]:
# model_BoW = GaussianNB(priors=priors)
model_BoW = MultinomialNB()
# model_BoW = GridSearchCV(model, parameters)

In [90]:
model_BoW.fit(X_train_bow.toarray(), y_train)

MultinomialNB()

2. TF-IDF Representation

In [91]:
# model_tfidf = GaussianNB(priors=priors)
model_tfidf = MultinomialNB()
# model_tfidf = GridSearchCV(model, parameters)

In [92]:
model_tfidf.fit(X_train_tfidf.toarray(), y_train)

MultinomialNB()

## Performance Metric

Test the above model with F1 score

In [93]:
from sklearn.metrics import f1_score

1. BoW Representation

In [94]:
y_pred_BoW = model_BoW.predict(X_test_bow.toarray())
score_BoW = f1_score(y_test, y_pred_BoW, average='macro')

2. TF-IDF Representation

In [95]:
y_pred_tfidf = model_tfidf.predict(X_test_tfidf.toarray())
score_tfidf = f1_score(y_test, y_pred_tfidf, average='macro')

Finally, compare the two F1 scores

In [96]:
print(f"F1 Score for BoW Naive-Bayes model: {score_BoW}")
print(f"F1 Score for TF-IDF Naive-Bayes model: {score_tfidf}")

# Normal Gaussian Model with Priors
# F1 Score for BoW Naive-Bayes model: 0.4691528243483322
# F1 Score for TF-IDF Naive-Bayes model: 0.5077292892355515

# Multinomial NB
# F1 Score for BoW Naive-Bayes model: 0.6821318078269769
# F1 Score for TF-IDF Naive-Bayes model: 0.5438953105865063

# GridSearchCV for Gaussian NB (30+ minutes)
# F1 Score for BoW Naive-Bayes model: 0.4704550456140844
# F1 Score for TF-IDF Naive-Bayes model: 0.5077292892355515

F1 Score for BoW Naive-Bayes model: 0.6821318078269769
F1 Score for TF-IDF Naive-Bayes model: 0.5438953105865063


MultinomialNB seems to perform better than GaussianNB. We'll try to optimize the hyperparameter of MultinomialNB using GridSearchCV

In [97]:
from sklearn.metrics import make_scorer

f1_macro_scorer = make_scorer(f1_score, average='macro')

parameters = {
    'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],
    'fit_prior': [True, False]
}

bow_grid = GridSearchCV(MultinomialNB(), parameters, cv=5, scoring=f1_macro_scorer)
tfidf_grid = GridSearchCV(MultinomialNB(), parameters, cv=5, scoring=f1_macro_scorer)

In [98]:
bow_grid.fit(X_train_bow, y_train)
model_bow = bow_grid.best_estimator_

tfidf_grid.fit(X_train_tfidf, y_train)
model_tfidf = tfidf_grid.best_estimator_

In [99]:
y_pred_bow = model_bow.predict(X_test_bow)
score_bow = f1_score(y_test, y_pred_bow, average='macro')

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)
score_tfidf = f1_score(y_test, y_pred_bow, average='macro')

In [100]:
print(f"F1 Score for hyper-optimized BoW Multinomial Naive-Bayes model: {score_bow}")
print(f"F1 Score for hyper-optimized TF-IDF Multinomial Naive-Bayes model: {score_tfidf}")

F1 Score for hyper-optimized BoW Multinomial Naive-Bayes model: 0.7044097669097669
F1 Score for hyper-optimized TF-IDF Multinomial Naive-Bayes model: 0.7044097669097669
